In [1]:
import numpy as np
import pandas as pd

In [2]:
import http.client, urllib.request, urllib.parse, urllib.error, base64
import json
import time, sys

In [3]:
sub_key = 'd7e2d15af06a4bafb8ea55b4c2b468f2'

In [4]:
microwave = pd.read_csv('microwave.tsv', sep='\t')
hair_dryer = pd.read_csv('hair_dryer.tsv', sep='\t')
pacifier = pd.read_csv('pacifier.tsv', sep='\t')

In [5]:
def gen_body(content):
    body = {
      "documents": [
        {
          "id": "1",
          "text": content
        }
      ]
    }
    body = json.dumps(body)
    return body

In [6]:
def get_sentiment(content):
    body = gen_body(content)
    headers = {
        # Request headers
        'Content-Type': 'application/json',
        'Ocp-Apim-Subscription-Key': sub_key,
    }

    params = urllib.parse.urlencode({
    })

    try:
        conn = http.client.HTTPSConnection('chinaeast2.api.cognitive.azure.cn')
        conn.request("POST", "/text/analytics/v2.0/sentiment?%s" % params, body, headers)
        response = conn.getresponse()
        data = response.read()
        # print(data)
        dict = json.loads(data)
        conn.close()
        return dict
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))

In [7]:
def get_sentiment_score(content):
    dict = get_sentiment(content)
    # print(dict)
    return dict['documents'][0]['score']

In [8]:
score = get_sentiment_score(microwave.iloc[np.random.randint(0,1615)]['review_body'])
print(score)

0.8590556383132935


In [9]:
def preprocess(df):
    df.drop('marketplace', axis=1, inplace=True)
    df.drop('product_category', axis=1, inplace=True)
    df['content_score'] = 0.0
    # df.head()

In [10]:
def add_score(df):
    preprocess(df)
    for index, row in df.iterrows():
        # print(row['review_body'])
        try:
            df['content_score'][index] = get_sentiment_score(row['review_body'])
            # print(df['content_score'][index])
        except IndexError as e:
            df['content_score'][index] = -1
        time.sleep(0.1)

In [11]:
add_score(microwave)
microwave.head()

/home/minaduki/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/minaduki/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,customer_id,review_id,product_id,product_parent,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,content_score
0,21879631,RY52KZABZK8QF,B0052G14E8,423421857,danby 0.7 cu.ft. countertop microwave,1,0,0,N,Y,Do not recommend it. go use your money for som...,3 of the buttons stopped working after a month...,8/31/2015,0.234928
1,14964566,R3GCOEV4HYZG2I,B0055UBB4O,423421857,danby 0.7 cu.ft. countertop microwave,5,0,0,N,Y,Looks as good as the picture,Didn't use it long so I can't attest to that b...,8/31/2015,0.121529
2,13230389,R1V2OPPNL0QGCE,B0052G14E8,423421857,danby 0.7 cu.ft. countertop microwave,4,0,0,N,Y,Four Stars,"Very nice microwave, great price",8/31/2015,0.998171
3,43655888,R9Q0QDTLKV567,B004ZU09QQ,423421857,danby 0.7 cu.ft. countertop microwave,3,0,1,N,Y,quiet,"Quiet, but does not seem like 1000 watt power.",8/31/2015,0.500000
4,117794,R3DL7HYC3QTWNI,B005GSZB7I,827502283,"whirlpool stainless look countertop microwave,...",4,0,0,N,Y,Four Stars,It's bigger than I thought.,8/31/2015,0.500000


In [12]:
microwave.to_csv('microwave_scored.csv')

In [13]:
add_score(hair_dryer)
hair_dryer.head()

/home/minaduki/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/minaduki/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,customer_id,review_id,product_id,product_parent,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,content_score
0,34678741,R9T1FE2ZX2X04,B003V264WW,732252283,remington ac2015 t|studio salon collection pea...,5,0,0,N,Y,Works great,Works great!,8/31/2015,0.836403
1,11599505,RE36JAD5V53PO,B0009XH6V4,670161917,andis micro turbo hair dryer,4,0,0,N,Y,I love travel blow dryers because they are eas...,"This dries my hair faster that bigger, more po...",8/31/2015,0.088761
2,2282190,RIDHM8B7SCCV3,B0007NZPY6,16483457,conair pro hair dryer,5,0,1,N,Y,Five Stars,Love this dryer!,8/31/2015,0.994075
3,43669858,R14QGWPCHU9LSE,B00BB8ZIW0,253917972,remington silk ceramic professional hair dryer,5,0,0,N,Y,Five Stars,styling hair in style,8/31/2015,0.859669
4,107098,R35BHQJHXXJD59,B003V264WW,732252283,remington ac2015 t|studio salon collection pea...,4,0,0,N,N,I think's great. The cord length is perfect,I just got this last week. I think's great. Th...,8/31/2015,0.803617


In [14]:
hair_dryer.to_csv('hair_dryer_scored.csv')

In [15]:
add_score(pacifier)
pacifier.head()

/home/minaduki/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/minaduki/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,customer_id,review_id,product_id,product_parent,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,content_score
0,40626522,R1A3ZUBR8TSAKY,B00793CZAE,572944212,"mary meyer wubbanub plush pacifier, lamb",5,0,0,N,Y,Love this!,Perfect match for the Gund Huggybuddy I bought...,8/31/2015,0.946600
1,16290022,RLJNYBK4FGBYX,B003PCYMP4,911821018,wubbanub lamb infant pacifier,5,0,0,N,Y,Love 💕,My little girl love this paci contraption!,8/31/2015,0.891480
2,10216509,R26QCW75C4JDOK,B003CK3LDI,392768822,wubbanub infant pacifier - giraffe,5,0,0,N,Y,Five Stars,My son loves this one and will only sleep if h...,8/31/2015,0.811086
3,114040,R2E7N0TVLUHUDR,B003CK3LDI,392768822,wubbanub infant pacifier - giraffe,5,0,0,N,Y,Five Stars,Perfect,8/31/2015,0.973931
4,27971579,R1SO9VMCIGZX3U,B003PCYMP4,911821018,wubbanub lamb infant pacifier,5,0,0,N,Y,Five Stars,Amazing addition to the nursery!,8/31/2015,0.900886


In [16]:
pacifier.to_csv('pacifier_scored.csv')

In [108]:
microwave.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1615 entries, 2015-08-31 to 2004-06-19
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_id        1615 non-null   int64  
 1   review_id          1615 non-null   object 
 2   product_id         1615 non-null   object 
 3   product_parent     1615 non-null   int64  
 4   product_title      1615 non-null   object 
 5   star_rating        1615 non-null   int64  
 6   helpful_votes      1615 non-null   int64  
 7   total_votes        1615 non-null   int64  
 8   vine               1615 non-null   object 
 9   verified_purchase  1615 non-null   object 
 10  review_headline    1615 non-null   object 
 11  review_body        1615 non-null   object 
 12  review_date        1615 non-null   object 
 13  content_score      1615 non-null   float64
 14  all_score          1615 non-null   float64
dtypes: float64(2), int64(5), object(8)
memory usage: 201.9

In [107]:
microwave = microwave.set_index(pd.to_datetime(microwave['review_date']))

In [109]:
microwave.head()

,customer_id,review_id,product_id,product_parent,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,content_score,all_score
review_date,,,,,,,,,,,,,,,
2015-08-31,21879631,RY52KZABZK8QF,B0052G14E8,423421857,danby 0.7 cu.ft. countertop microwave,1,0,0,N,Y,Do not recommend it. go use your money for som...,3 of the buttons stopped working after a month...,8/31/2015,0.234928,0.198332
2015-08-31,14964566,R3GCOEV4HYZG2I,B0055UBB4O,423421857,danby 0.7 cu.ft. countertop microwave,5,0,0,N,Y,Looks as good as the picture,Didn't use it long so I can't attest to that b...,8/31/2015,0.121529,0.140624
2015-08-31,13230389,R1V2OPPNL0QGCE,B0052G14E8,423421857,danby 0.7 cu.ft. countertop microwave,4,0,0,N,Y,Four Stars,"Very nice microwave, great price",8/31/2015,0.998171,0.997596
2015-08-31,43655888,R9Q0QDTLKV567,B004ZU09QQ,423421857,danby 0.7 cu.ft. countertop microwave,3,0,1,N,Y,quiet,"Quiet, but does not seem like 1000 watt power.",8/31/2015,0.500000,0.500000
2015-08-31,117794,R3DL7HYC3QTWNI,B005GSZB7I,827502283,"whirlpool stainless look countertop microwave,...",4,0,0,N,Y,Four Stars,It's bigger than I thought.,8/31/2015,0.500000,0.500000


In [111]:
len(hair_dryer['review_body'][273])

76

In [95]:
def add_score_concated(df):
    df['all_score'] = 0.0
    for index, row in df.iterrows():
        # print(row['review_body'])
        try:
            df['all_score'][index] = get_sentiment_score(row['review_headline'] + '. ' + row['review_body'])
            # print(row['review_headline'] + '. ' + row['review_body'])
            # print(df['content_score'][index])
        except IndexError as e:
            df['all_score'][index] = -1
        time.sleep(0.1)

In [38]:
add_score_concated(microwave)
microwave.head()

/home/minaduki/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/minaduki/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,customer_id,review_id,product_id,product_parent,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,content_score,all_score
0,21879631,RY52KZABZK8QF,B0052G14E8,423421857,danby 0.7 cu.ft. countertop microwave,1,0,0,N,Y,Do not recommend it. go use your money for som...,3 of the buttons stopped working after a month...,8/31/2015,0.234928,0.198332
1,14964566,R3GCOEV4HYZG2I,B0055UBB4O,423421857,danby 0.7 cu.ft. countertop microwave,5,0,0,N,Y,Looks as good as the picture,Didn't use it long so I can't attest to that b...,8/31/2015,0.121529,0.140624
2,13230389,R1V2OPPNL0QGCE,B0052G14E8,423421857,danby 0.7 cu.ft. countertop microwave,4,0,0,N,Y,Four Stars,"Very nice microwave, great price",8/31/2015,0.998171,0.997596
3,43655888,R9Q0QDTLKV567,B004ZU09QQ,423421857,danby 0.7 cu.ft. countertop microwave,3,0,1,N,Y,quiet,"Quiet, but does not seem like 1000 watt power.",8/31/2015,0.500000,0.500000
4,117794,R3DL7HYC3QTWNI,B005GSZB7I,827502283,"whirlpool stainless look countertop microwave,...",4,0,0,N,Y,Four Stars,It's bigger than I thought.,8/31/2015,0.500000,0.500000


In [39]:
microwave.to_csv('microwave_scored_all.csv')

In [92]:
hair_dryer.isnull().sum()

Unnamed: 0           0
customer_id          0
review_id            0
product_id           0
product_parent       0
product_title        0
star_rating          0
helpful_votes        0
total_votes          0
vine                 0
verified_purchase    0
review_headline      2
review_body          0
review_date          0
content_score        0
dtype: int64

In [93]:
hair_dryer = hair_dryer.fillna(' ')

In [94]:
hair_dryer.isnull().sum()

Unnamed: 0           0
customer_id          0
review_id            0
product_id           0
product_parent       0
product_title        0
star_rating          0
helpful_votes        0
total_votes          0
vine                 0
verified_purchase    0
review_headline      0
review_body          0
review_date          0
content_score        0
dtype: int64

In [96]:
add_score_concated(hair_dryer)
hair_dryer.head()

/home/minaduki/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


KeyboardInterrupt: 

In [ ]:
hair_dryer.to_csv('hair_dryer_scored_all.csv')

In [ ]:
pacifier.isnull().sum()

In [ ]:
pacifier = pacifier.fillna(' ')

In [ ]:
pacifier.isnull().sum()

In [ ]:
add_score_concated(pacifier)
pacifier.head()

In [ ]:
pacifier.to_csv('pacifier_scored_all.csv')

In [81]:
hair_dryer.fillna(0)

,customer_id,review_id,product_id,product_parent,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,content_score,all_score
0,34678741,R9T1FE2ZX2X04,B003V264WW,732252283,remington ac2015 t|studio salon collection pea...,5,0,0,N,Y,Works great,Works great!,8/31/2015,0.836403,0.836403
1,11599505,RE36JAD5V53PO,B0009XH6V4,670161917,andis micro turbo hair dryer,4,0,0,N,Y,I love travel blow dryers because they are eas...,"This dries my hair faster that bigger, more po...",8/31/2015,0.088761,0.148486
2,2282190,RIDHM8B7SCCV3,B0007NZPY6,16483457,conair pro hair dryer,5,0,1,N,Y,Five Stars,Love this dryer!,8/31/2015,0.994075,0.993318
3,43669858,R14QGWPCHU9LSE,B00BB8ZIW0,253917972,remington silk ceramic professional hair dryer,5,0,0,N,Y,Five Stars,styling hair in style,8/31/2015,0.859669,0.845091
4,107098,R35BHQJHXXJD59,B003V264WW,732252283,remington ac2015 t|studio salon collection pea...,4,0,0,N,N,I think's great. The cord length is perfect,I just got this last week. I think's great. Th...,8/31/2015,0.803617,0.860929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11465,35599734,R2O50YNP83CG34,B000065DJY,919751065,revlon 1875w volumizing hair dryer,5,14,18,N,N,great hair!,"I just purchased this dryer, and I really like...",8/21/2002,0.186152,0.000000
11466,43977044,R2JQPUYU65C4QD,B00005O0MZ,694290590,conair corp pers care 146np conair ionic condi...,1,26,32,N,N,Disappointment with dryer,I purchased it because it was supposed to be q...,8/13/2002,0.024459,0.000000
11467,37733836,R3GO6L5PWBS0IW,B00005O0MZ,694290590,conair corp pers care 146np conair ionic condi...,5,4,4,N,Y,The best dryer!!!,"I love this dryer not only is it QUIET, it rea...",7/13/2002,0.916666,0.000000
11468,38947355,R3JMGN42OJCL97,B00005O0MZ,694290590,conair corp pers care 146np conair ionic condi...,5,78,87,N,N,Excellent for fine/limp hair,This hairdryer far exceeded my expection for<B...,4/20/2002,0.803010,0.000000


In [78]:
hair_dryer.loc[273].fillna(0)

customer_id                                                   51177255
review_id                                               R2TW4FSXQ60M75
product_id                                                  B001UE7D2I
product_parent                                               127343313
product_title        andis 1875-watt tourmaline ceramic ionic styli...
star_rating                                                          5
helpful_votes                                                        0
total_votes                                                          0
vine                                                                 N
verified_purchase                                                    Y
review_headline                                                      0
review_body          Used once - recently cut hair - Sure to use it...
review_date                                                   8/9/2015
content_score                                                      0.5
all_sc

In [83]:
hair_dryer['review_headline'].isnull().sum()

2

In [82]:
hair_dryer.loc[273]

customer_id                                                   51177255
review_id                                               R2TW4FSXQ60M75
product_id                                                  B001UE7D2I
product_parent                                               127343313
product_title        andis 1875-watt tourmaline ceramic ionic styli...
star_rating                                                          5
helpful_votes                                                        0
total_votes                                                          0
vine                                                                 N
verified_purchase                                                    Y
review_headline                                                    NaN
review_body          Used once - recently cut hair - Sure to use it...
review_date                                                   8/9/2015
content_score                                                      0.5
all_sc

In [72]:
hair_dryer['review_headline'][273]

nan

In [60]:
hair_dryer['review_headline'][273] + '. ' + hair_dryer['review_body'][273]

TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [28]:
rand = np.random.randint(0,1615)
str = microwave.iloc[rand]['review_headline'] + '. ' + microwave.iloc[rand]['review_body']
print(str)

It's tinny.. It's kind of tinny.  Probably just too small . The latch for closing the door sticks so you need to slam the it,  and the whole thing shakes.  I'm already prepared for itt o die, and I will buy a larger and more expensive one.


In [18]:
print(str)

Three StarsAs described


In [36]:
microwave.head()

,customer_id,review_id,product_id,product_parent,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,content_score,all_score
0,21879631,RY52KZABZK8QF,B0052G14E8,423421857,danby 0.7 cu.ft. countertop microwave,1,0,0,N,Y,Do not recommend it. go use your money for som...,3 of the buttons stopped working after a month...,8/31/2015,0.234928,0.198332
1,14964566,R3GCOEV4HYZG2I,B0055UBB4O,423421857,danby 0.7 cu.ft. countertop microwave,5,0,0,N,Y,Looks as good as the picture,Didn't use it long so I can't attest to that b...,8/31/2015,0.121529,0.140624
2,13230389,R1V2OPPNL0QGCE,B0052G14E8,423421857,danby 0.7 cu.ft. countertop microwave,4,0,0,N,Y,Four Stars,"Very nice microwave, great price",8/31/2015,0.998171,0.997596
3,43655888,R9Q0QDTLKV567,B004ZU09QQ,423421857,danby 0.7 cu.ft. countertop microwave,3,0,1,N,Y,quiet,"Quiet, but does not seem like 1000 watt power.",8/31/2015,0.500000,0.500000
4,117794,R3DL7HYC3QTWNI,B005GSZB7I,827502283,"whirlpool stainless look countertop microwave,...",4,0,0,N,Y,Four Stars,It's bigger than I thought.,8/31/2015,0.500000,0.500000


In [ ]:
microwave.iloc[np.random.randint(0,1615)]['review_body']

In [ ]:
len(microwave.product_id.unique())

In [ ]:
microwave.total_votes.max()

In [ ]:
microwave.info()